# Callbacks

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline
%load_ext tensorboard

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
os.listdir('dataset/cifar/')

['labels.txt', 'test', 'train']

In [6]:
train_paths = glob('dataset/cifar/train/*.png')[:1000]  # 보기용으로 데이터 갯수를 1000개로 제한
test_paths = glob('dataset/cifar/test/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

In [7]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [8]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [9]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [10]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [11]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [12]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [14]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Callbacks

In [15]:
starttime = datetime.now()
starttime

datetime.datetime(2021, 3, 29, 17, 38, 15, 247583)

In [16]:
datetime.now().strftime("%Y%m%d-%H%M%S")

'20210329-173815'

In [17]:
logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))

In [18]:
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=logdir,
    write_graph=True,
    write_images=True,
    histogram_freq=1

)

In [19]:
%tensorboard --logdir logs --port 8008

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 18052.

https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/r2/image_summaries.ipynb#scrollTo=IJNpyVyxbVtT

## Training

http://localhost:6006

In [20]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard]
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
 1/31 [..............................] - ETA: 0s - loss: 2.2967 - accuracy: 0.0625WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
31/31 [==============================] - 1s 32ms/step - loss: 2.3142 - accuracy: 0.0940 - val_loss: 2.2962 - val_accuracy: 0.1512
Epoch 2/10
31/31 [==============================] - 1s 22ms/step - loss: 2.2849 - accuracy: 0.1405 - val_loss: 2.2797 - val_accuracy: 0.1341
Epoch 3/10
31/31 [==============================] - 1s 22ms/step - loss: 2.1834 - accuracy: 0.1808 - val_loss: 2.1360 - val_accuracy: 0.2389
Epoch 4/10
31/31 [==============================] - 1s 22ms/step - loss: 2.1558 - accuracy: 0.1849 - val_loss: 2.1451 - val_accuracy: 0.2

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup

### LambdaCallback

In [21]:
# https://www.tensorflow.org/tensorboard/r2/image_summaries

import sklearn.metrics
import itertools
import io

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [22]:
test_images, test_labels = next(iter(test_dataset))  # Confusion Matrix 그릴 때 필요한 Test Image


def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [24]:
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [25]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard, cm_callback]
)

Epoch 1/10
31/31 [==============================] - 1s 39ms/step - loss: 1.8583 - accuracy: 0.3068 - val_loss: 1.8115 - val_accuracy: 0.3246
Epoch 2/10
31/31 [==============================] - 1s 35ms/step - loss: 1.7597 - accuracy: 0.3533 - val_loss: 1.8213 - val_accuracy: 0.3367
Epoch 3/10
31/31 [==============================] - 1s 35ms/step - loss: 1.7479 - accuracy: 0.3616 - val_loss: 1.7890 - val_accuracy: 0.3317
Epoch 4/10
31/31 [==============================] - 1s 34ms/step - loss: 1.7306 - accuracy: 0.3419 - val_loss: 1.7673 - val_accuracy: 0.3196
Epoch 5/10
31/31 [==============================] - 1s 37ms/step - loss: 1.7193 - accuracy: 0.3709 - val_loss: 1.7424 - val_accuracy: 0.3619
Epoch 6/10
31/31 [==============================] - 1s 35ms/step - loss: 1.6490 - accuracy: 0.3905 - val_loss: 1.7567 - val_accuracy: 0.3528
Epoch 7/10
31/31 [==============================] - 1s 34ms/step - loss: 1.6952 - accuracy: 0.3790 - val_loss: 1.7675 - val_accuracy: 0.3669
Epoch 8/10
31

## Expert

In [26]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [27]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [28]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)  # Loss 계산
    gradients = tape.gradient(loss, model.trainable_variables)  # 모델의 trainable_variable을 하여금 loss를 통해 기울기를 얻음
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # 구한 최적화된 값을 variable에 적용

    train_loss(loss)
    train_accuracy(labels, predictions)

In [29]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [30]:
logdir = os.path.join('logs',  datetime.now().strftime("%Y%m%d-%H%M%S"))

In [31]:
file_writer = tf.summary.create_file_writer(logdir)

In [ ]:
for epoch in range(num_epochs):
    for step, (images, labels) in enumerate(train_dataset):            
        train_step(images, labels)

    with file_writer.as_default():
        tf.summary.image('input_image', images, step=step)

    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))